In [46]:
import os

from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

from langchain_openai import ChatOpenAI
llm_model = 'gpt-4o-mini'
chat_model = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
from langchain_experimental.utilities import PythonREPL # tool로 사용하면 에러가 나옴.
python_repl = PythonREPL()
python_repl.run("print(3.1**2.1)")

'10.761171606099687\n'

In [44]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_experimental.tools import PythonREPLTool

python_tool = PythonREPLTool()


def print_and_execute(code):
    print("작성된 코드:")
    print(code)
    print("실행 결과:")
    return python_tool.invoke(code)

# 파이썬 코드를 작성하도록 요청하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 파이썬 프로그래머이며 초절정 고수야. 짧고 간결하고 훌륭한 코드를 작성해줘.
            코드를 작성할 때 코드에 대한 설명, 주석 등 불필요한 거 다 필요없고 오직 코드만 출력해줘.
            """,
        ),
        ("human", "{input}"),
    ]
)

# 프롬프트와 LLM 모델을 사용하여 체인 생성
chain = prompt | chat_model | StrOutputParser() | RunnableLambda(print_and_execute)

# 결과 출력
print(chain.invoke("피보나치 수열의 10번째까지 수들을 리스트 형태로 출력하는 코드를 작성해줘."))

작성된 코드:
```python
fibonacci = [0, 1]
for i in range(2, 10):
    fibonacci.append(fibonacci[i-1] + fibonacci[i-2])
print(fibonacci)
```
실행 결과:
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]



In [43]:
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
def top5_results(query):
    return search.results(query, 5)


tool = Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=top5_results,
)

tool.run("2024년 한국 계엄령 관련 뉴스")

[{'title': '비상계엄: 과거 역사 굴곡 겪었던 세대부터 MZ까지 이들이 보는 2024 ...',
  'link': 'https://www.bbc.com/korean/articles/c623dwznlj4o',
  'snippet': '3 days ago ... 3일 밤 급작스러운 윤석열 대통령의 비상계엄 선포에 한국 시민들은 혼란스러운 시간을 보냈다. 늦은 밤 뉴스를 보다 놀라 잠을 제대로 자지 못했다는\xa0...'},
 {'title': 'VOA 한국어 - YouTube',
  'link': 'https://www.youtube.com/@VOAKorea',
  'snippet': "긴장감 속 '비상계엄' 주시 - 2024년 12월 4일. VOA 한국어. 9.3K views. 3 days ago ... [VOA 뉴스] '한국 민주 회복력' 신뢰…'정치 문제 해결' 잘할 것. 621 views."},
 {'title': '윤석열 대통령이 갑작스럽게 비상계엄을 선포한 이유 - BBC News ...',
  'link': 'https://www.bbc.com/korean/articles/c75w6rn5kw0o',
  'snippet': '3 days ago ... 1987년 이후 초유 사태. 2024년 12월 4일. 한국 대통령이 계엄령을 선포한 ... TOP 뉴스. 1. 윤석열 대통령은 왜 갑작스럽게 비상계엄을 선포했을까.'},
 {'title': '북미 한국학연구소장들 “윤석열 대통령 계엄선포 규탄… 한국 시민과 ...',
  'link': 'https://www.voakorea.com/a/7889327.html',
  'snippet': '2 days ago ... 하버드, 스탠퍼드, 조지워싱턴 등 북미 13개 대학의 한국학연구소장들이 4일 성명을 내고 “2024년 12월 3일 오후 10시 24분, 한국 윤석열 대통령은 계엄을\xa0...'},
 {'title': '윤석열, 스캔들에 휩싸였다 계엄령까지 선포한 한국의 대통령 - BBC ...',
  '

In [49]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False,
    include_domains=["naver.com"],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

tool.invoke({"query": "2024년 한국 비상계엄령에 대한 뉴스"})

[{'url': 'https://blog.naver.com/PostView.naver?blogId=jkim0815&logNo=223681764179',
  'content': '2024년 12월 4일 뉴스 검색 ... (서울=연합뉴스) 김승욱 기자 = 비상계엄 해제 이후 윤석열 대통령의 첫 공개 일정이었던 4일 오전 마약류 대응상황 점검회의가 순연됐다. ... 주한 美대사 "계엄령에 놀라…한국 민주주의 회복력에 고무돼"'},
 {'url': 'https://blog.naver.com/PostView.naver?blogId=g1goon&logNo=223685341000',
  'content': '[2024.12.07] 12.3 계엄령에 대한 소고 ... 심지어 2024년 승리한 대선 역시도 부정선거가 있었다고 주장했다. ... 〈자료사진=연합뉴스〉 비상계엄 당시 중앙선거관리위원회에 계엄군이 대거 투입된 것을 두고 김용현 전 국방부 장관은 "부정선거 의혹을 수사하려 했다'},
 {'url': 'https://in.naver.com/angelina/contents/internal/765698267976192',
  'content': '윤석열 비상계엄령에 . ... 국민의 안전을 지키기 위해 2024년 12월 3일 23:00부로 대한민국 전역에 다음 사항을 포고합니다. 1. 국회와 지방의회, 정당의 활동과 정치적 결사, 집회, 시위 등 일체의 정치활동을 금한다. ... 가짜뉴스, 여론조작, 허위선동을 금한다.'},
 {'url': 'https://m.blog.naver.com/moonjji63/223681909262',
  'content': "2024년 12월 4일 부동산 경제 뉴스. ... lh '최근 주거 이슈에 대한 국민인식' 연구 국민 80%, 주거 불평등 심각해 인식 lh, 주거 불평등 지수 개발 평생 집을 사지 못할 것이라고 지레 포기하는 사람이 10명 중 4명에 달하는 것으로 나타났다. ... 尹계엄령에…환율 1.66%"},
 {'url': 'htt